# Dog Cat Classifier - Pretrained - pytroch

## if using google colab

### Connect Colab to Google drive 

In [51]:
# from google.colab import drive
# drive.mount._DEBUG = True
# # drive.mount('/content/drive', force_remount=True)
# drive.mount('/content/drive')

# #Change the Directry to your project in GOogle driver (Create a folder named : "App")
# %cd "/content/drive/My Drive/Colab Notebooks"

# # List driver
# !ls


### Download data set from kaggle
[Downloading Kaggle Datasets into Google Colab](https://medium.com/@opalkabert/downloading-kaggle-datasets-into-google-colab-fb9654c94235)

In [52]:


# !pip install -U -q kaggle
# !mkdir -p ~/.kaggle

# from google.colab import files
# # files.upload()

# !cp kaggle.json ~/.kaggle/
# # !kaggle datasets list

# !kaggle datasets download -d chetankv/dogs-cats-images
# !ls

### Check GPU availability

In [53]:
import torch
import numpy as np

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [54]:
import torch
import os
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from torchvision import datasets, transforms, models

torch.set_printoptions(linewidth=120)  # Displa optin for output
torch.set_grad_enabled(True) # Already on by default

batch_size = 32

In [55]:
print(torch.__version__)
print(torchvision.__version__)

1.1.0
0.3.0


In [56]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

### Preprocess images

In [57]:

data_dir = '../input/dog vs cat/dataset'

transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.Resize(size=(224,224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

# test_transform = transforms.Compose([
#                                 transforms.Resize(254),
#                                 transforms.CenterCrop(254),
#                                 transforms.ToTensor()])


train_data =  torchvision.datasets.ImageFolder(data_dir + '/training_set', transform = transform)
test_data =  torchvision.datasets.ImageFolder(data_dir + '/test_set', transform = transform)

In [60]:
BATCH_SIZE = 64

train_iterator = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=BATCH_SIZE)
valid_iterator = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE)
# test_iterator = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE)

In [61]:
device = torch.device('cuda')

In [62]:
import torchvision.models as models

model = models.resnet18(pretrained=True).to(device)

In [63]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [64]:
for param in model.parameters():
    param.requires_grad = False

In [65]:
model.fc = nn.Linear(in_features=512, out_features=2).to(device)

In [67]:

criterion = nn.CrossEntropyLoss()

In [68]:
optimizer = optim.Adam(model.parameters())

In [69]:
def calculate_accuracy(fx, y):
    preds = fx.max(1, keepdim=True)[1]
    correct = preds.eq(y.view_as(preds)).sum()
    acc = correct.float()/preds.shape[0]
    return acc

In [70]:
def train(model, device, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for (x, y) in iterator:
        
        x = x.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
                
        fx = model(x)
        
        loss = criterion(fx, y)
        
        acc = calculate_accuracy(fx, y)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [71]:
def evaluate(model, device, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for (x, y) in iterator:

            x = x.to(device)
            y = y.to(device)

            fx = model(x)

            loss = criterion(fx, y)

            acc = calculate_accuracy(fx, y)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [72]:
EPOCHS = 10
SAVE_DIR = 'models'
MODEL_SAVE_PATH = os.path.join(SAVE_DIR, 'resnet18-dogs-vs-cats.pt')

best_valid_loss = float('inf')

if not os.path.isdir(f'{SAVE_DIR}'):
    os.makedirs(f'{SAVE_DIR}')

for epoch in range(EPOCHS):
    train_loss, train_acc = train(model, device, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, device, valid_iterator, criterion)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), MODEL_SAVE_PATH)
    
    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:05.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:05.2f}% |')

| Epoch: 01 | Train Loss: 0.206 | Train Acc: 92.21% | Val. Loss: 0.110 | Val. Acc: 96.09% |
| Epoch: 02 | Train Loss: 0.113 | Train Acc: 95.65% | Val. Loss: 0.084 | Val. Acc: 96.92% |
| Epoch: 03 | Train Loss: 0.094 | Train Acc: 96.31% | Val. Loss: 0.089 | Val. Acc: 96.68% |
| Epoch: 04 | Train Loss: 0.091 | Train Acc: 96.35% | Val. Loss: 0.082 | Val. Acc: 96.83% |
| Epoch: 05 | Train Loss: 0.087 | Train Acc: 96.71% | Val. Loss: 0.084 | Val. Acc: 96.73% |
| Epoch: 06 | Train Loss: 0.082 | Train Acc: 96.85% | Val. Loss: 0.075 | Val. Acc: 97.41% |
| Epoch: 07 | Train Loss: 0.075 | Train Acc: 97.05% | Val. Loss: 0.073 | Val. Acc: 97.12% |
| Epoch: 08 | Train Loss: 0.077 | Train Acc: 96.96% | Val. Loss: 0.076 | Val. Acc: 97.02% |
| Epoch: 09 | Train Loss: 0.078 | Train Acc: 96.71% | Val. Loss: 0.062 | Val. Acc: 97.41% |
| Epoch: 10 | Train Loss: 0.074 | Train Acc: 97.19% | Val. Loss: 0.058 | Val. Acc: 98.10% |


In [73]:
model.load_state_dict(torch.load(MODEL_SAVE_PATH))

test_loss, test_acc = evaluate(model, device, valid_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:05.2f}% |')

| Test Loss: 0.062 | Test Acc: 97.75% |
